In [29]:
!pip install wandb

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     |████████████████████████████████| 1.9 MB 4.6 MB/s            
     |████████████████████████████████| 168 kB 105.1 MB/s            
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 182 kB 79.1 MB/s            
     |████████████████████████████████| 62 kB 2.7 MB/s             
     |████████████████████████████████| 140 kB 75.2 MB/s            
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=23348fe6a424decbe9e8043d38f208adfc94f6bf464cbec9ec52484c9bb4db86
  Stored in directory: /aiffel/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtool

In [ ]:
# mecab 설치
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [1]:
!pip install kogpt2_transformers
!pip install rouge

In [2]:
# 라이브러리 불러오기

import torch
import torch.nn as nn
from tqdm import tqdm
from torch.utils.data import Dataset
from kogpt2_transformers import get_kogpt2_model, get_kogpt2_tokenizer
import csv
import pandas as pd
from datasets import Dataset
from rouge import Rouge
import re
import os

In [49]:
# Category별 DataFrame을 출력 (train)
train = pd.read_csv('/aiffel/aiffel/aiffelthon/csv/train.tsv', delimiter = '\t')
Category = train['Category'].unique()
data_path = '/aiffel/aiffel/aiffelthon/csv/'
def train_categori_ext(data):
    df = pd.DataFrame()
    for c in Category:
        df = pd.concat([df, data[data['Category'] == c].iloc[0:int(len(data[data['Category'] == c])*0.05)]], axis = 0)
        df.reset_index(inplace=True, drop=True)
        df.to_csv(data_path+'train_5%.tsv', sep = '\t', encoding = 'utf-8', index = False)
    return df

# 데이터 파일 생성

train_categori_ext(train)

,Id,Text,Summary,Category
0,fd321028-d5b4-55f7-9e20-2eaa262f9154,"['그럼 날짜는 가격 큰 변동 없으면 6.28-7.13로 확정할까?', '우리 비행...","비행기 표 가격에 대해 이야기하며, 특가 이벤트를 기다리고 있다.",상거래(쇼핑)
1,c51be2e4-c8d0-5cea-b1ae-cde1fe8f8ab6,"['Kf마스크만 5부제 하는거지?', '응. 면마스크는 아무때나 사도될껀?', '면...",비염이 있어서 싸게 나온 일회용 부직포 마스크를 사두려고 한다.,상거래(쇼핑)
2,e90e721f-00d1-5114-aa5d-5f1061472a29,['아 근데 케이크 업체들 봤는데 중앙동쪽 거기는 맛만있고 디자인은 그냥그런것같애'...,케이크 업체 중 중앙동 쪽은 맛만 있고 디자인은 별로고 고잔동 케이크 업체는 배달도...,상거래(쇼핑)
3,b215f3a2-d647-59f9-8410-1274ee5edd97,"['칫솔사야하는데 쓱으로 살까?', '뭘 칫솔사는것까지 물어보시남ㅋㅋㅋ', '아 그...",칫솔을 3개월에 하나씩 바꿔서 왕 칫솔 사러 신세계(쓱) 가자고 했다.,상거래(쇼핑)
4,0bda61b6-1396-5a2a-a049-0b4035e40d59,"['잠도안오네ㅐ얼릉 고구마츄 먹고싶단', '그게 그렇게 맛있었어??? 아주 여보 빼...",잠도 안 와서 고구마 말랭이를 양심상 하나만 먹으려고 한다.,상거래(쇼핑)
...,...,...,...,...
13989,94a9b0e9-5a84-5739-9851-f5bae80d936d,"['옷 오늘간대', '오 드디어 저녁에 입어야지 #@이모티콘#', '예상저녁7시~9...",배송 예상 시간이 7-9시인데 옷을 입고 나오라고 했다.,행사
13990,5eed65c3-f9a9-5736-a400-ca79f43516fe,"['함안올끼가?', '#@이름# 데꼬? ㅎ 일주일요양가까', '어', '이번주말고 ...",이번 주에 비 소식이 있어 다음 주에 가기로 한다.,행사
13991,6505168e-cccb-5bdf-8cc9-19b488f6b8cf,"['언니 안뇽! 잘지냉?', '#@이름#아!!! 히힛 난 잘지내! 너는 잘지내구이쪄...",8월 첫째 주가 휴가라서 시간이 되면 만나자고 했다.,행사
13992,3dd63c1d-22d3-5720-b30e-d9d6fd81d6f5,"['그럼 거기그때 진흥아파트??', '네넨 진흥아파트에요~', '오키', '올라오시...",1시 반 넘어서 만나기로 하고 진흥아파트 동 호수를 알려주었다.,행사


In [50]:
# Category별 DataFrame을 출력
val = pd.read_csv('/aiffel/aiffel/aiffelthon/csv/val.tsv', delimiter = '\t')
Category = val['Category'].unique()
data_path = '/aiffel/aiffel/aiffelthon/csv/'
def val_categori_ext(data):
    df = pd.DataFrame()
    for c in Category:
        df = pd.concat([df, data[data['Category'] == c].iloc[0:int(len(data[data['Category'] == c])*0.05)]], axis = 0)
        df.reset_index(inplace=True, drop=True)
        df.to_csv(data_path+'val_5%.tsv', sep = '\t', encoding = 'utf-8', index = False)
    return df

# 데이터 파일 생성

val_categori_ext(val)

,Id,Text,Summary,Category
0,9effd65e-e18f-5510-8866-d6a677314c9c,"['웅', '영업팀과장님이 보내줬는데 팀장님이 해줄지 모르겠다 저번에 부산갈때도 숙...",팀장님이 출장 가서 머물 숙소를 계속해서 더 싼 데로 하게 한다고 이야기하고 있다.,일과 직업
1,49b9f9b7-a79d-5283-9553-95b8f156ff90,"['너는 잘가라....회사.... 선택 잘해..', '알겠어 ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ 많...",이제 이력서를 쓰고 영어도 해야 한다고 해서 첫 회사를 잘 들어가라고 했다.,일과 직업
2,df6b6bac-648b-5d32-86a3-9b8b215e5f95,"['느낌상 대통령까지는 아니고 오시면 여사님정도오시지않을까 ㅋㅋㅋ', '그러면서',...",느낌상 대통령까지는 아니고 오시면 여사님 정도 오시지 않을까라며 이에 대해 이야기하...,일과 직업
3,3043293f-7bb5-5edb-9b1f-85120e305c3d,"['숨만수이ㅓ도 숨만쉬어도 100 이내', '한달안에 일 무조건 해야대', '아 딱...",한 달 안에 무조건 일을 시작해서 돈을 벌어야 한다.,일과 직업
4,2b9b9c39-2721-50e6-af3e-e5c00138f43d,"['목요일은 외근이구 금요일은 출장!!!', '금요일이 당진이양?', '아닝아닝 1...",목요일에 외근이고 금요일에 출장인데 당진은 10일에 간다.,일과 직업
...,...,...,...,...
1741,b03b9ad0-62fa-54a5-b552-69049bd642ee,"['엄마 점심식사 뭐먹었어요??', '우린 중국집 시켜먹었다', '오오 짜장면 탕수...","엄마에게 오늘 점심 뭐 먹었냐고 묻고 엄마가 중국집에서 볶음밥, 잡채밥 시켜 먹었다...",식음료
1742,ec83f40a-3c01-581d-962c-80fd62a5c888,"['매운거', '담에 점심 엽떡', '자신있음', '엽떡 먹는 사람', '오 저색기...",엽떡(엽기 떡볶이) 제일 매운맛을 3명에서 도전하기로 했다.,식음료
1743,2b8dad32-cb6a-5d2a-89c4-06ba5f8dc8f8,"['아 술마시고싶다 취할줄알았는데 ㅋㅋㅋㅋㅋㅋ', '누룩오자 누룩 군고구마막걸리 존...",군고구마막걸리가 맛있다고 하니 빨리 마셔보고 싶다고 한다.,식음료
1744,bd803ef5-c166-54ad-8d96-ac73fb055b3d,"['종촌동에서 뭘 주워먹어야겠다... 어른들 저녁먹고 가신대 뭐사먹지,.....',...",어른들이 저녁 먹고 가신다고 해서 뭘 사 먹을까 하니 근처에 국수나무가 맛있다고 한다.,식음료


In [70]:
# torch cash 초기화

import gc
gc.collect()
torch.cuda.empty_cache()

Model : KoGPT-2

Parametors size : 

[How to use GPT2LMHeadModel for conditional generation](https://github.com/huggingface/transformers/issues/970)

Data NAME : Ai_hub 대화문 요약 Data Set

In [3]:
# 데이터 전처리

def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열 (.) 제거
    sentence = re.sub(r'[ㄱ-ㅎㅏ-ㅣ.,?!]+[/ㄱ-ㅎㅏ-ㅣ.,?!]', '', sentence) # 여러개 자음, 모음, 구두점에서 마지막 1개만 남긴다
    sentence = re.sub("[^가-힣a-z0-9-.,!?]", " ", sentence) # 지정한 문자 제외 공백으로 전환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거

    return sentence

In [56]:
# 데이터 로드 및 전처리와 메타데이터 적용 

def data_load(data_path = '/aiffel/aiffel/aiffelthon/csv/train_5%.tsv'):
    data = []
    train_df = pd.read_csv(data_path, delimiter = '\t')
    #train_df = train_df.iloc[::2, :] # 짝수 row data만 추출한다.
    #train_df.reset_index(drop=True, inplace = True) # 인덱스 초기화
    
    # 전처리 적용
    clean_text = []
    
    for s in tqdm(train_df['Text']):
        clean_text.append(preprocess_sentence(s))
                          
    train_df['Text'] = clean_text
    
    clean_headlines = []
                          
    for s in tqdm(train_df['Summary']):
        clean_headlines.append(preprocess_sentence(s))
        
    train_df['Summary'] = clean_headlines 
    
    # 메타데이터 적용
#     def metadata_train(train_df): 
#         meta = '#'+train_df['Category']+'# '
#         train_df['Text'] = meta + train_df['Text']
#         return train_df['Text']
    
#     metadata_train(train_df)
    
    # Series > list
    train_df['Text'].tolist()
    train_df['Summary'].tolist()
    
    return train_df['Text'], train_df['Summary']

def val_load(data_path = '/aiffel/aiffel/aiffelthon/csv/val_5%.tsv'):
    val_df = pd.read_csv(data_path, delimiter = '\t')
    #val_df = val_df.iloc[::2, :]
    #val_df.reset_index(drop=True, inplace = True)
    

    # 전처리 적용
    clean_text_val = []
                          
    for s in tqdm(val_df['Text']):
        clean_text_val.append(preprocess_sentence(s))
    
    val_df['Text'] = clean_text_val
    
    clean_headlines_val = []
                          
    for s in tqdm(val_df['Summary']):
        clean_headlines_val.append(preprocess_sentence(s))
        
    val_df['Summary'] = clean_headlines_val 
    
    # 메타데이터 적용
#     def metadata_val(val_df): 
#         meta = '#'+val_df['Category']+'# '
#         val_df['Text'] = meta + val_df['Text']
#         return val_df['Text']
    
#     metadata_val(val_df)
                          
    # Series > list
    val_df['Text'].tolist()
    val_df['Summary'].tolist()
    
    return val_df['Text'], val_df['Summary']

In [16]:
data_1 = data_load()

100%|██████████| 55993/55993 [00:00<00:00, 96673.03it/s]


In [17]:
data_1

(0        #상거래(쇼핑)# 그럼 날짜는 가격 큰 변동 없으면 6.28-7.13로 확정할까? ...
 1        #상거래(쇼핑)# kf마스크만 5부제 하는거지? , 응. 면마스크는 아무때나 사도될...
 2        #상거래(쇼핑)# 아 근데 케이크 업체들 봤는데 중앙동쪽 거기는 맛만있고 디자인은 ...
 3        #상거래(쇼핑)# 칫솔사야하는데 쓱으로 살까? , 뭘 칫솔사는것까지 물어보시남 , ...
 4        #상거래(쇼핑)# 잠도안오네 얼릉 고구마츄 먹고싶단 , 그게 그렇게 맛있었어 아주 ...
                                ...                        
 55988    #행사# 일본으로 낚시가 , 그건또첨인데 맛있는거만 먹으러다녔었지 낚시는 한번두 ,...
 55989    #행사# 이모부생신27일 그래서 투썸케이크 방금전화로예약함 , 이름 이생일은 28일...
 55990    #행사# 이게 무슨일 십삼만원 확인해보니까 다낭가는게 4만 돌아오는게 6만이네 취소...
 55991    #행사# 병원 갔다가 칸다소바나 갈까 혼자 점심은 어디서 좋을까 , 칸다소바 가라 ...
 55992    #행사# 겨론식끝나쪄용 , 이열 오늘 언니결혼이였꾸나꺅 축하드립니다 오나애 화장이랑...
 Name: Text, Length: 55993, dtype: object,
 0                     비행기 표 가격에 대해 이야기하며, 특가 이벤트를 기다리고 있다.
 1                      비염이 있어서 싸게 나온 일회용 부직포 마스크를 사두려고 한다.
 2        케이크 업체 중 중앙동 쪽은 맛만 있고 디자인은 별로고 고잔동 케이크 업체는 배달도...
 3                     칫솔을 3개월에 하나씩 바꿔서 왕 칫솔 사러 신세계 가자고 했다.
 4                        잠도 안 와서 고구마 말랭이를 양심상 하나만 먹으려고 한

In [139]:
data_2 = val_load()

100%|██████████| 17502/17502 [00:00<00:00, 95907.57it/s]


In [140]:
type(data_1)

tuple

In [142]:
data_2

(0        #일과 직업# 웅 , 영업팀과장님이 보내줬는데 팀장님이 해줄지 모르겠다 저번에 부산...
 1        #일과 직업# 느낌상 대통령까지는 아니고 오시면 여사님정도오시지않을까 , 그러면서 ...
 2        #일과 직업# 목요일은 외근이구 금요일은 출장 , 금요일이 당진이양? , 아닝아닝 ...
 3        #일과 직업# 메일 주소 알려주세여 제가 점심먹고와서 보낼게여 , 넵넵 시스템 사진...
 4        #일과 직업# 나 크리스마스떄도 일한다는데 시급 1.5배 맞나요 , 어 1.5배 요...
                                ...                        
 17497    #식음료# 마카롱은 , 우리 들어오자마자 우걱우걱 , 2개 정도 먹어야함 후 자제하...
 17498    #식음료# 저번에 스벅에서 바움쿠헨 먹었거둥 나원래 바움쿠헨 좋아하긴하거든 근데 저...
 17499    #식음료# 배고프닭 5시반에 시키면 5시 40분에 주문 , 언니씻고 뭐하면 , 맵구...
 17500    #식음료# 닭갈비 맛있었다 맛나맛나 그티웅 , 아깐 먹기 싫다더니 , 역시 너의 선...
 17501    #식음료# 아! 본죽에 신메뉴 나온 거 알아 , 오 신메뉴 나온 걸 왜 못봤지 ? ...
 Name: Text, Length: 17502, dtype: object,
 0           팀장님이 출장 가서 머물 숙소를 계속해서 더 싼 데로 하게 한다고 이야기하고 있다.
 1        느낌상 대통령까지는 아니고 오시면 여사님 정도 오시지 않을까라며 이에 대해 이야기하...
 2                         목요일에 외근이고 금요일에 출장인데 당진은 10일에 간다.
 3           점심을 먹고 와서 알려준 메일 주소로 롯데 원티비 큐에이 담당자에게 메일을 보냈다.
 4        근로계약서를 써야한다고 하며 크리스마스 때 일하는 것의 시급을 1.5배해달라고 할 .

### hyparam은 Code를 가져올 때 설정을 유지중입니다. GPT-2  학습이 제대로 이루어진게 어제부터라서 수정하지는 못했습니다.

### 그래도 수정을 한 부분은 문장을 생성할 때 eos_token, pad_token이 생성되어 이 부분을 출력하지 않도록 bad_words_index를 사용했습니다.

In [46]:
# Model 제작

import torch
import torch.nn as nn
from torch.utils.data import Dataset
from kogpt2_transformers import get_kogpt2_model


class AbstractiveKoGPT2(nn.Module):
    def __init__(self):
        super(AbstractiveKoGPT2, self).__init__()
        self.kogpt2 = get_kogpt2_model()

    def generate(self,
                 input_ids,
                 do_sample=True,
                 max_length= 128,
                 num_beams=5,
                 #top_p=0.92, # top_p, k는 code를 가져올 때 설정을 유지하였고 보내주신 텍스트 생성
                 #top_p=50,
                 temperature= 0.6, # 0.6 or 0.8 
                 no_repeat_ngram_size = None, # 변경 시도해보기
                 num_return_sequences=3,
                 early_stopping=False,
                 bad_words_ids = [1, 3]
                 ):
        return self.kogpt2.generate(input_ids,
                                     #do_sample = do_sample,
                                     max_length = max_length,
                                     num_beams = num_beams, # 더 많은 글 작성 예시 생성
                                     #top_p = top_p, # 글의 표현 범위 조절
                                     #top_k = top_k, # 글의 표현 범위 조절
                                     temperature = temperature, # 글의 창의성 조절
                                     no_repeat_ngram_size= no_repeat_ngram_size,
                                     num_return_sequences = num_return_sequences,
                                     early_stopping = early_stopping,
                                     eos_token_id = 1,
                                     pad_token_id = 3,
                                    )

    def forward(self, input, labels = None):
        if labels is not None:
            outputs = self.kogpt2(input, labels=labels)
        else:
            outputs = self.kogpt2(input)

        return outputs

### 스페셜 토큰은 당시 대회 1등을 수상한 팀이 사전에 파악해둔 라벨링 텍스트를 토대로 가져왔으며, 정규표현식을 사용해 해당 텍스트가 데이터에 포함된 것을 확인했습니다.

In [8]:
# 학습용 데이터 세트 제작

class AbstrativeDataset(Dataset):
    def __init__(self,
                 #n_ctx = 1024, 
                 n_ctx = 256, 
                 texts_max_length = 128,
                 summary_max_length = 64,
                 device = 'cpu'
                 ):
        
        self.data =[]
        self.device = device
        self.tokenizer = get_kogpt2_tokenizer()

        bos_token_id = [self.tokenizer.bos_token_id] # , 0
        eos_token_id = [self.tokenizer.eos_token_id] # , 1
        pad_token_id = [self.tokenizer.pad_token_id] # , 3
        
        special_words = [
        "#@주소#", "#@이모티콘#", "#@이름#", "#@URL#", "#@소속#",
        "#@기타#", "#@전번#", "#@계정#", "#@url#", "#@번호#", "#@금융#", "#@신원#",
        "#@장소#", "#@시스템#사진#", "#@시스템#동영상#", "#@시스템#기타#", "#@시스템#검색#",
        "#@시스템#지도#", "#@시스템#삭제#", "#@시스템#파일#", "#@시스템#송금#", "#@시스템#",]
        
        
        tokenizer.add_special_tokens({"additional_special_tokens": special_words}) 
        # 스페셜 토큰은 적용

        datas = data_load()
        
        for texts, text_sum in tqdm(zip(datas[0],datas[1])):
            text = texts
            abstractive_summary = text_sum

#             tmp_str =''
#             for article in articles:
#                 tmp_str += article

            # encode
            # truncate, if string exceed max length
            enc_tmp_str = self.tokenizer.encode(text, truncation= True, max_length=texts_max_length, add_special_tokens=False)
            enc_abstractive_summary = self.tokenizer.encode(abstractive_summary, truncation= True, max_length=summary_max_length)
            
            #  요약할 문장  요약된 문장 
            index_of_words = bos_token_id + enc_tmp_str+ eos_token_id + enc_abstractive_summary + eos_token_id
            pad_token_len = n_ctx - len(index_of_words)
            index_of_words += pad_token_id * pad_token_len
            #print(f'max: {max(index_of_words)}, min: {min(index_of_words)}')
            
            # RuntimeError: CUDA error: device-side assert triggered
            if max(index_of_words)<50000:
                self.data.append(torch.tensor(index_of_words).to(device))

    def __len__(self):
        return len(self.data)

    def __getitem__(self,index):
        item = self.data[index]
        return item

In [23]:
data_3 = AbstrativeDataset()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
100%|██████████| 139996/139996 [00:01<00:00, 95352.29it/s]
139996it [00:54, 2554.39it/s]


In [24]:
# 데이터 셋이 제대로 만들어졌는지 확인 완료

len(data_3)

139996

In [67]:
# 데이터 셋이 제대로 만들어지지 않은 경우

len(dataset)

0

In [9]:
# 검증용 데이터 세트 제작
class EvalAbstrativeDataset(Dataset):
    def __init__(self,
                 device,
                 tokenizer,
                 #n_ctx = 1024,
                 n_ctx = 256,
                 data_path = '/aiffel/aiffel/aiffelthon/csv/val_5%.tsv',
                 texts_max_length = 128,
                 summary_max_length = 64,
                 ):
        #self.data = pd.DataFrame()
        self.data =[]
        self.tokenizer = tokenizer
        print(device)
        
        bos_token_id = [self.tokenizer.bos_token_id] # 
        eos_token_id = [self.tokenizer.eos_token_id] # 

        #json_datas = json_load(data_path=data_path)
        #eval_datas = data_load(data_path=data_path)

        special_words = [
        "#@주소#", "#@이모티콘#", "#@이름#", "#@URL#", "#@소속#",
        "#@기타#", "#@전번#", "#@계정#", "#@url#", "#@번호#", "#@금융#", "#@신원#",
        "#@장소#", "#@시스템#사진#", "#@시스템#동영상#", "#@시스템#기타#", "#@시스템#검색#",
        "#@시스템#지도#", "#@시스템#삭제#", "#@시스템#파일#", "#@시스템#송금#", "#@시스템#",]
        
        tokenizer.add_special_tokens({"additional_special_tokens": special_words})
        
        datas = val_load()
        
        for data in tqdm(datas[0]):
            #id = dict_data['id']
            texts = data

            tmp_str =''
            for text in texts:
                tmp_str += text

            # encode
            # truncate, if string exceed max length
            enc_tmp_str = self.tokenizer.encode(tmp_str, truncation = True, max_length=texts_max_length, add_special_tokens=False)

            #  요약할 문장  요약된 문장 
            index_of_words = bos_token_id + enc_tmp_str+ eos_token_id

            self.data.append({
                #'id':id,
                'input':torch.tensor([index_of_words]).to(device)
            })

    def __len__(self):
        return len(self.data)

    def __getitem__(self,index):
        item = self.data[index]
        return item

### def_seed부분은 아직 이해가 부족하여 수정은 하지않았습니다.

### 현재 1 epoch로 모든 train data(29만개)를 사용할 시 약 5시간이 걸립니다. 데이터를 분할하여 다양한 방법을 적용해보는 것을 목표로하고 성능 향상이 보인다면 전체 데이터를 다 사용할 예정입니다.

### 데이터 분할 기준은 각 Category에서 20%씩 추출하여 사용해볼 생각입니다.

In [ ]:
# train

import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import display
from torch.utils.data import dataloader
import random
#import wandb

# 토크나이저 불러오기

tokenizer = get_kogpt2_tokenizer()

def set_seed(seed = 42): # 난수 생성 seed 고정
    torch.backends.cudnn.deterministic = True # Deterministic 한 알고리즘만 사용하게 하기
    torch.backends.cudnn.benchmark = False # cudnn.benchmar 활성화 여부
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
        
if __name__ == '__main__':
    
    set_seed()
    #wandb.init(project= 'bakbak', entity= 'quarter2000', name= f'KG')
    #checkpoint_path = "/aiffel/aiffel/aiffelthon/checkpoint_2/"
    #save_ckpt_path = f"{checkpoint_path}/kogpt2-abstractive_2.pth"
    checkpoint_path = "/aiffel/aiffel/aiffelthon/checkpoint_2"
    save_ckpt_path = f"{checkpoint_path}/kogpt2-abstractive.pth"
    
    n_epoch = 3         # Num of Epoch
    #batch_size = 4      # 배치 사이즈
    batch_size = 8      # 배치 사이즈
    device = "cuda" if torch.cuda.is_available() else "cpu"
    #device = 'cpu'
    print('device: ',device)
    save_step = 50 # 학습 저장 주기
    learning_rate = 5e-5  # Learning Rate

    dataset= AbstrativeDataset(device=device)
    train_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    # checkpoint = torch.load(save_ckpt_path,map_location=torch.device(device))
    model = AbstractiveKoGPT2()
    model.to(device)
    # model.load_state_dict(checkpoint['model_state_dict'])

    loss_fct = torch.nn.CrossEntropyLoss(ignore_index=3)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    total_losses = []
    losses =[]

    if os.path.isfile(save_ckpt_path):
        checkpoint = torch.load(save_ckpt_path, map_location=device)
        pre_epoch = checkpoint['epoch']
        pre_loss = checkpoint['loss']
        total_losses = checkpoint['losses']

        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        print(f"load pretrain from: {save_ckpt_path}, epoch={pre_epoch}, loss={pre_loss}")
        

    for epoch in tqdm(range(n_epoch)):
        count = 0
        with tqdm(total=len(train_loader), desc=f"Train({epoch})") as pbar:
            for i, data in enumerate(train_loader):
                optimizer.zero_grad()
#                 for d in data:
#                     print(max(d),min(d))
                outputs = model(data, labels=data)
                _, logits = outputs[:2]

                # Shift so that tokens < n predict n
                shift_logits = logits[..., :-1, :].contiguous()
                shift_labels = data[..., 1:].contiguous()

                loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
                loss.backward()
                optimizer.step()

                losses.append(loss.item())

                if (count > 0 and count % save_step == 0) or (len(data) < batch_size):
                    torch.save({
                        'epoch': epoch,
                        'train_no': count,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'loss': loss,
                        'losses': losses
                    }, save_ckpt_path)
                count += 1
                pbar.update(1)
                pbar.set_postfix_str(f"Loss: {loss.item():.3f} ({np.mean(losses):.3f})")
                #if i%50==0:
                    #wandb.log({'loss':np.mean(losses)})

        total_losses.append(np.mean(losses))

#     # data
    data = {
        "loss": total_losses
    }
    df = pd.DataFrame(data)
    display(df)

#     # graph
#     plt.figure(figsize=[12, 4])
#     plt.plot(losses, label="loss")
#     plt.legend()
#     plt.xlabel('Epoch')
#     plt.ylabel('Loss')
#     plt.show()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


device:  cuda


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
100%|██████████| 13994/13994 [00:00<00:00, 88699.45it/s]
13994it [00:05, 2583.55it/s]
Train(2):  76%|███████▋  | 1336/1750 [18:25<03:52,  1.78it/s, Loss: 3.876 (4.312)]

In [54]:
# test

tokenizer = get_kogpt2_tokenizer()
ckpt_path = '/aiffel/aiffel/aiffelthon/checkpoint/kogpt2-abstractive.pth'
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

eval_datas = EvalAbstrativeDataset(tokenizer=tokenizer,device=device,data_path='/aiffel/aiffel/aiffelthon/csv/val_5%.tsv')

checkpoint = torch.load(ckpt_path,map_location=torch.device(device))
model = AbstractiveKoGPT2()
model.to(device)
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()

my_summaries=[]
# for data in tqdm(eval_datas[:3337]):
#     input_ids = data['input']
#     sample_output = model.generate(input_ids=input_ids,max_length=1024)
#     summary = tokenizer.decode(sample_output[0].tolist()[len(input_ids[0]):-1])
#     summary = summary.replace('','').replace('','')
#     my_summaries.append(summary)

for data in tqdm(eval_datas):
    input_ids = data['input']
    max_toc = max(input_ids[0])
    min_toc = min(input_ids[0])
    if max_toc<50000:
        sample_output = model.generate(input_ids=input_ids, max_length=256)
        summary = tokenizer.decode(sample_output[0].tolist()[len(input_ids[0]):-1], skip_special_tokens=True) 
        # 스페셜 토큰은 출력되지 않게 제한을 걸었습니다.
        summary = summary.replace('','').replace('','')
        my_summaries.append(summary)
    else:
        summary = tokenizer.decode(input_ids[0])
        summary = summary.replace('','').replace('','').replace('','')
        my_summaries.append(summary)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


cuda
cuda


100%|██████████| 1746/1746 [09:40<00:00,  3.01it/s]


In [8]:
# 요약 데이터 형태 확인

type(my_summaries)

list

In [33]:
# 실제 원문과 요약문 불러오기

Input_Text = val_load()[0]
real_Sum = val_load()[1]

100%|██████████| 1746/1746 [00:00<00:00, 83110.19it/s]


In [106]:
# 2 에포크 결과

for num in range(0,10):
    print('Text : ', Input_Text[num])
    print('real_summarization : ', real_Sum[num])
    print('predict_summarization : ', my_summaries[num])
    print('-'*120)

Text :  #일과 직업# 웅 , 영업팀과장님이 보내줬는데 팀장님이 해줄지 모르겠다 저번에 부산갈때도 숙소로 엄청 싸워서 , 웅 흥 4개월가는거도아니고 3일가는데 좀해주지 거 얼마한다구 , 내말이 아니 해봤자 3일 다 합해도 몇만원 차이인데 너무해 그때는 2일이었는데도 만원 더 싼데에서 자꾸 자라고 넘 시러 , 일단
real_summarization :  팀장님이 출장 가서 머물 숙소를 계속해서 더 싼 데로 하게 한다고 이야기하고 있다.
predict_summarization :  영업팀 과장님이 보내준 부산 갈 때도 숙소로 엄청 싸웠는데 3일 가는데 2만 더 싼 데서 자꾸 자라고 너무 시러한다.
------------------------------------------------------------------------------------------------------------------------
Text :  #일과 직업# 너는 잘가라회사 선택 잘해 , 알겠어 많이 힘들구나 나도 이제 이력서쓰고 영어도 해야해 , 우리 똥갱애지 화이팅 첫 회사 잘들어가야한다! , 좋은 시절 다 지났다
real_summarization :  이제 이력서를 쓰고 영어도 해야 한다고 해서 첫 회사를 잘 들어가라고 했다.
predict_summarization :  잘 가라고 회사 선택을 잘 하라고 하고 영어도 해야 한다고 한다.
------------------------------------------------------------------------------------------------------------------------
Text :  #일과 직업# 느낌상 대통령까지는 아니고 오시면 여사님정도오시지않을까 , 그러면서 , 샘 여기있었구낭 , 종이강사랑 자기가 옆에서 서포트 한다는 식으로 , 벌써 시나리오가 , 어르신들 못하는 설명 자기가 한다는 식으로 , 그럼 본인이 다 하시면 되겠넹 , 그니까요
real_summarization :  느낌상 대통령

In [37]:
# 6 에포크 결과

for num in range(0, 5):
    print('Text : ', Input_Text[num])
    print('real_summarization : ', real_Sum[num])
    print('predict_summarization : ', my_summaries[num])
    print('-'*120)

Text :  #일과 직업# 웅 , 영업팀과장님이 보내줬는데 팀장님이 해줄지 모르겠다 저번에 부산갈때도 숙소로 엄청 싸워서 , 웅 흥 4개월가는거도아니고 3일가는데 좀해주지 거 얼마한다구 , 내말이 아니 해봤자 3일 다 합해도 몇만원 차이인데 너무해 그때는 2일이었는데도 만원 더 싼데에서 자꾸 자라고 넘 시러 , 일단
real_summarization :  팀장님이 출장 가서 머물 숙소를 계속해서 더 싼 데로 하게 한다고 이야기하고 있다.
predict_summarization :  영업팀 과장님이 보내준 부산 갈 때도 숙소로 엄청 싸워서 3일 동안 갔는데 2만 원 차이에서 자꾸 자라고 너무 시러
------------------------------------------------------------------------------------------------------------------------
Text :  #일과 직업# 너는 잘가라회사 선택 잘해 , 알겠어 많이 힘들구나 나도 이제 이력서쓰고 영어도 해야해 , 우리 똥갱애지 화이팅 첫 회사 잘들어가야한다! , 좋은 시절 다 지났다
real_summarization :  이제 이력서를 쓰고 영어도 해야 한다고 해서 첫 회사를 잘 들어가라고 했다.
predict_summarization :  잘 간다고 회사 선택을 하고 영어도 해야 한다.
------------------------------------------------------------------------------------------------------------------------
Text :  #일과 직업# 느낌상 대통령까지는 아니고 오시면 여사님정도오시지않을까 , 그러면서 , 샘 여기있었구낭 , 종이강사랑 자기가 옆에서 서포트 한다는 식으로 , 벌써 시나리오가 , 어르신들 못하는 설명 자기가 한다는 식으로 , 그럼 본인이 다 하시면 되겠넹 , 그니까요
real_summarization :  느낌상 대통령까지는 아니고 오시면

In [45]:
# 6 에포크 결과

for num in range(0, 5):
    print('Text : ', Input_Text[num])
    print('real_summarization : ', real_Sum[num])
    print('predict_summarization : ', my_summaries[num])
    print('-'*120)

Text :  #일과 직업# 웅 , 영업팀과장님이 보내줬는데 팀장님이 해줄지 모르겠다 저번에 부산갈때도 숙소로 엄청 싸워서 , 웅 흥 4개월가는거도아니고 3일가는데 좀해주지 거 얼마한다구 , 내말이 아니 해봤자 3일 다 합해도 몇만원 차이인데 너무해 그때는 2일이었는데도 만원 더 싼데에서 자꾸 자라고 넘 시러 , 일단
real_summarization :  팀장님이 출장 가서 머물 숙소를 계속해서 더 싼 데로 하게 한다고 이야기하고 있다.
predict_summarization :  영업팀 과장님이 보내준 부산 갈 때도 숙소로 많이 싸웠는데 3일 동안 가느라 2만 원 차이가 나는 데서 자라고 너무 시러 그렇다.
------------------------------------------------------------------------------------------------------------------------
Text :  #일과 직업# 너는 잘가라회사 선택 잘해 , 알겠어 많이 힘들구나 나도 이제 이력서쓰고 영어도 해야해 , 우리 똥갱애지 화이팅 첫 회사 잘들어가야한다! , 좋은 시절 다 지났다
real_summarization :  이제 이력서를 쓰고 영어도 해야 한다고 해서 첫 회사를 잘 들어가라고 했다.
predict_summarization :  잘 가라고 회사 선택을 잘 하라고 응원해 주고 있다.
------------------------------------------------------------------------------------------------------------------------
Text :  #일과 직업# 느낌상 대통령까지는 아니고 오시면 여사님정도오시지않을까 , 그러면서 , 샘 여기있었구낭 , 종이강사랑 자기가 옆에서 서포트 한다는 식으로 , 벌써 시나리오가 , 어르신들 못하는 설명 자기가 한다는 식으로 , 그럼 본인이 다 하시면 되겠넹 , 그니까요
real_summarization :  느낌상 대통령

In [49]:
# 실제 요약문 Reference로 불러오기

Reference = val_load()[1][:100]


100%|██████████| 1746/1746 [00:00<00:00, 95826.53it/s]


### rouge.get_scores 사용 방법

- rouge.get_scores(Model이 요약한 문장, 실제 요약문, avg = True)

In [104]:
# Rouge-Score 출력 - 3epoch
# num_beams = 5 적용
# num_return_sequences = 3 적용
# 검증용 데이터 5%건에 대한 Rooge-Score의 평균

Reference = Reference
Confernce = my_summaries
rouge = Rouge()
rouge.get_scores(Reference, Confernce, avg=True)

{'rouge-1': {'r': 0.21979430827267368,
  'p': 0.16328730990389997,
  'f': 0.1805112093216261},
 'rouge-2': {'r': 0.07069632558817938,
  'p': 0.052375325649611566,
  'f': 0.05738430670451539},
 'rouge-l': {'r': 0.2124911362408781,
  'p': 0.15764003968932178,
  'f': 0.174334298253391}}

In [14]:
# Rouge-Score 출력 - 3epoch
# num_beams = 5 적용
# num_return_sequences = 3 적용
# 검증용 데이터 5%건에 대한 Rooge-Score의 평균

Reference = Reference
Confernce = my_summaries
rouge = Rouge()
rouge.get_scores(Reference, Confernce, avg=True)

{'rouge-1': {'r': 0.22844666725142712,
  'p': 0.1625996046590448,
  'f': 0.18265204179178565},
 'rouge-2': {'r': 0.07692712648999303,
  'p': 0.0539825795682743,
  'f': 0.060403850651108135},
 'rouge-l': {'r': 0.21980814195128878,
  'p': 0.15646141045418605,
  'f': 0.17572315861174834}}

In [27]:
# Rouge-Score 출력 - 3epoch
# do_sample = True 적용
# top_k = 50 적용
# top_p = 0.92 적용
# num_return_sequences = 3 적용
# temperature = 0.8
# 검증용 데이터 5%건에 대한 Rooge-Score의 평균

Reference = Reference
Confernce = my_summaries
rouge = Rouge()
rouge.get_scores(Reference, Confernce, avg=True)

{'rouge-1': {'r': 0.22844666725142712,
  'p': 0.1625996046590448,
  'f': 0.18265204179178565},
 'rouge-2': {'r': 0.07692712648999303,
  'p': 0.0539825795682743,
  'f': 0.060403850651108135},
 'rouge-l': {'r': 0.21980814195128878,
  'p': 0.15646141045418605,
  'f': 0.17572315861174834}}

In [32]:
# Rouge-Score 출력 - 6epoch
# num_beams = 5 적용
# num_return_sequences = 3 적용
# 검증용 데이터 5%건에 대한 Rooge-Score의 평균

Reference = Reference
Confernce = my_summaries
rouge = Rouge()
rouge.get_scores(Reference, Confernce, avg=True)

{'rouge-1': {'r': 0.19402062215709237,
  'p': 0.15448623137288045,
  'f': 0.16549857750123848},
 'rouge-2': {'r': 0.0553530354192373,
  'p': 0.04419058959117343,
  'f': 0.04664070772620241},
 'rouge-l': {'r': 0.18485717631485127,
  'p': 0.1472732153558843,
  'f': 0.1576700673408802}}

In [44]:
# Rouge-Score 출력 - 6epoch
# num_beams = 5 적용
# num_return_sequences = 3 적용
# no_repeat_ngram_size = 3 적용
# 검증용 데이터 5%건에 대한 Rooge-Score의 평균

Reference = Reference
Confernce = my_summaries
rouge = Rouge()
rouge.get_scores(Reference, Confernce, avg=True)

{'rouge-1': {'r': 0.18164438192086513,
  'p': 0.13550023832431562,
  'f': 0.14977299216704743},
 'rouge-2': {'r': 0.04632337953323305,
  'p': 0.03419162666479835,
  'f': 0.03762267656653159},
 'rouge-l': {'r': 0.17379405085318006,
  'p': 0.12990577544040502,
  'f': 0.1434292194386092}}

In [53]:
# Rouge-Score 출력 - 9epoch
# num_beams = 5 적용
# num_return_sequences = 3 적용
# 검증용 데이터 5%건에 대한 Rooge-Score의 평균

reference = Reference.tolist()
model_out = my_summaries
rouge = Rouge()rouge.get_scores(model_out, reference, avg=True)

{'rouge-1': {'r': 0.14145790976533665,
  'p': 0.19073706848706845,
  'f': 0.1538319661821521},
 'rouge-2': {'r': 0.03928081026300841,
  'p': 0.054573315573315566,
  'f': 0.04158304378936719},
 'rouge-l': {'r': 0.13181306849549537,
  'p': 0.17913353313353317,
  'f': 0.14354718353560475}}

In [ ]:
# Rouge-Score 출력 - 9epoch
# num_beams = 5 적용
# num_return_sequences = 3 적용
# 검증용 데이터 5%건에 대한 Rooge-Score의 평균

reference = Reference.tolist()
model_out = my_summaries
rouge = Rouge()
score = rouge.get_scores(model_out, reference, avg=True)
type(score)